# Carousel mechanism design
Project: *LuSEE-Night* <br>
System: *Antenna Carousel* <br>
Author: *Joe Silber* <br>
Institution: *Lawrence Berkeley National Lab (LBNL)* <br>
Date: *July 2022* <br>

Abstract: Assessment of carousel (a.k.a. "turntable") mechanism and torques for the LuSEE-Night lunar instrument. The radio antennas rotate on a platform. Key design parameters and options for the rotation mechanism are considered here.

## Preliminaries

### Units and coordinate systems
Except where otherwise specified...
- all units are in meters, kilograms, newtons, and seconds
- all inertias and positions are with respect to a global coordinate system, with Z axis concentric to rotation axis

### Python initializations and units

In [1]:
import math
deg = '\u00b0'
Nm_per_ozin = 1 / 141.61193227806
m_per_in = 0.0254 
radsec_per_rpm = 2 * math.pi / 60  # i.e. rad/sec per rev/min
K_to_other = {'K': lambda T: T,
              'C': lambda T: T - 273.15,
              'F': lambda T: (T - 273.15)*9/5 + 32.0,
             }

### General material properties

In [2]:
rho_SS = 8000  # stainless steel density
rho_Ti = 4430  # titanium density
rho_Al = 2700  # aluminum density
rho_BeCu = 8300  # beryllium copper density
rho_Cu = 8900  # copper density
rho_plastic = 1300  # generic plastic density
E_Cu = 117e9 # copper modulus of elasticity
E_plastic = 2e9 # generic plastic modulus of elasticity
E_BeCu = 130e9  # beryllium copper modulus of elasticity
nu_BeCu = 0.3  # beryllium copper poisson ratio

### Carbon fiber properties

In [3]:
rho_f = 2170  # carbon fiber density, e.g. CN-80
rho_m = 1170  # resin matrix density, e.g. EX-1515
Mm = 0.32  # matrix mass fraction, i.e. prepreg resin content from vendor
Vv = 0.005  # as-cured void fraction
Vf = (1 - Vv) / (1 + rho_f / rho_m * Mm / (1 - Mm))  # fiber volume fraction, assuming no bleed (conservative)
cfrp_density = rho_f * Vf + rho_m * (1 - Vf - Vv)
print(f'Estimated carbon fiber reinforced plastic (CFRP) density = {cfrp_density:.1f} kg/m^3, volume fraction = {Vf*100:.1f}%')

Estimated carbon fiber reinforced plastic (CFRP) density = 1695.4 kg/m^3, volume fraction = 53.1%


### Other basic system properties

In [4]:
n_ant = 4  # number of antennas
rotation_range_deg = [-95, 95]  # degrees
rotation_range = [math.radians(x) for x in rotation_range_deg]  # radians
total_rotation_range = max(rotation_range) - min(rotation_range)

## Key mechanical design criteria
The design criteria below will be used in sizing components and ensuring sufficient mechanical factors of safey. These criteria are informed by key recommendations in NASA's [General Environmental Verification Standards (GEVS)](https://standards.nasa.gov/sites/default/files/standards/GSFC/B/0/gsfc-std-7000b_signature_cycle_04_28_2021_fixed_links.pdf) (accessed 2022-07-09).

### Structural load limit
The limit loads for the carousel mechanism are derived from GEVS Table 2.4-3,
*Generalized Random Vibration Test Levels*. For components weighing less than 22.7 kg, an overall qualification level (in $G_{rms}$) is directly specified. We then apply a proof factor per section 2.4.1.4.1, and a design factor to get the design limit. The design factor accommodated uncertainty in material properties and tolerances in the proof test method. These limits are applied for nominal accelerations in 3 orthogonal axes.

In [5]:
Grms_qual = 14.1
proof_factor = 1.25
design_factor = 1.2
Grms_design = Grms_qual * proof_factor * design_factor
print(f'Design load level = {Grms_design:.1f} G(rms)')

Design load level = 21.1 G(rms)


### Torque margin
Torque margin ($TM$) for the carousel turntable is assessed per GEVS section 2.4.5.3. Two safety factors, $FS_k$ and $FS_v$ are applied to the "known" and "variable" torques in the system. GEVS states:

> The minimum available driving torque for the mechanism shall be determined based on the FS listed above. The Torque Margin (TM) shall be greater than zero and shall be calculated using the following formula:

$$TM = \frac{T_{avail}}{FS_k \sum T_{known} + FS_v \sum T_{variable}} - 1$$

In the present study, the available driving torque $T_{avail}$ is considered at output of the the gearmotor (which is the component whose performance requirements we are aiming to specify). The torques are defined in GEVS as:

> **Driving Torques**
>
> $T_{avail}$ = Minimum Available Torque or Force generated by the mechanism at worst case environmental conditions at any time in its life. If motors are used in the system, $T_{avail}$ shall be determined at the output of the motor, not including gear heads or gear trains at its output based on minimum supplied motor voltage. $T_{avail}$ similarly applies to other actuators such as springs, pyrotechnics, solenoids, heat actuated devices, etc.
> 
> **Resistive Torques**
>
> $\sum T_{known}$ = Sum of the fixed torques or forces that are known and quantifiable such as
accelerated inertias ($T = I \alpha$) and not influenced by friction, temperature, life, etc. A constant Safety Factor is applied to the calculated torque.
>
> $\sum T_{variable}$ = Sum of the torques or forces that may vary over environmental conditions and life such as static or dynamic friction, alignment effects, latching forces, wire harness
loads, damper drag, variations in lubricant effectiveness, including degradation or depletion of lubricant over life, etc.

Here we use the highest (PDR phase) torque safety factor values from GEVS:

In [6]:
FSk = 2.0  # 2.0 at preliminary design review phase
           # drops to 1.5 at critical design review
           # remains 1.5 at acceptance / qualification testing

FSv = 4.0  # 4.0 at preliminary design review phase
           # drops to 3.0 at critical design review
           # drops to 2.0 at acceptance / qualification testing

### Thermal criteria
The carousel system must survive the lunar day/night cycle. It must operate at intermediate temperatures around dusk/dawn.

The plots below, showing lunar temperature as a function of hour and latitude, were retrieved July 2022 from an article (exact reference?) found through citations [here](https://quickmap.lroc.asu.edu/layers):

![Lunar temperatures as function of hour and latitude](figures/lunar-temperatures.png)

We anticipate LuSEE-Night will land at one of two sites:

1. $155^o$ E and $15^o$ S
2. $175^o$ E and $22^o$ S

These indicate that the lander will experience temperature extrema of:

In [7]:
temp_ranges = {'extrema': [90, 390]}  # Kelvin

#### Survival temperature range
For design and testing purposes, the survival temperature range is obtained by increasing the expected lunar range by a margin on either end.

In [8]:
temp_margin = 10.0  # deg C
temp_ranges['survival'] = [min(temp_ranges['extrema']) - temp_margin,
                           max(temp_ranges['extrema']) + temp_margin]

#### Operating temperature range
The operating temperature for carousel rotation will be less extreme. We want a range which is moderate enough to includes practical motors, transmissions, and bearings, yet wide enough to allow a significant period in the dusk/dawn time window where ground-based operators can effectively send commands, monitor the response, and make any adjustments if necessary. A good operating temperature range also offers practical conditions for testing in the lab.

***? Under the assumption of a perfluoropolyether (PFPE) based lubricant in the gears and bearings ... ?***

***? For what temperature range do we have equipment on hand to practically test operation of the motors, bearings, transmissions---the full turntable assembly---in vacuum ?***

Considering these factors, we propose an operating temperature range of 280 K - 320 K, which should provide generous time windows ($\sim$ 1 lunar hour $\approx$ 30 earth hours each), at lunar dawn and dusk.

In [9]:
temp_ranges['operating'] = [280, 320]

#### Temperature ranges summary

In [10]:
for name, temp_range in temp_ranges.items():
    for unit, func in K_to_other.items():
        converted = [func(T) for T in temp_range]
        print(f'Temperature range = {min(converted):+4.0f} to {max(converted):+4.0f} {deg}{unit} ({name})')
    print('')

Temperature range =  +90 to +390 °K (extrema)
Temperature range = -183 to +117 °C (extrema)
Temperature range = -298 to +242 °F (extrema)

Temperature range =  +80 to +400 °K (survival)
Temperature range = -193 to +127 °C (survival)
Temperature range = -316 to +260 °F (survival)

Temperature range = +280 to +320 °K (operating)
Temperature range =   +7 to  +47 °C (operating)
Temperature range =  +44 to +116 °F (operating)



## Loads
Total mass budget for the carousel system is:

In [11]:
mass_budget_total = 7.8  # kg

Some fraction of this mass (in particular the drive motor and transmission) will be static.

For the purposes of this document, we subdivide the rotating payload into the following estimated masses, moments, and transmission ratios.

### Turntable plate
Three common engineering materials are being considered for the turntable platter: aluminum, titanium, or carbon fiber reinforced plastic (CFRP).

In [12]:
plate_od = 0.500  # outer diameter
plate_id = 0.079  # inner diameter
plate_t = 0.010  # thickness
plate_r2, plate_r1 = plate_od/2, plate_id/2  # radii
plate_area = math.pi * (plate_r2**2 - plate_r1**2)
uniform_plate_inertia_over_mass = 0.5 * (plate_r1**2 + plate_r2**2)

Bending stiffness is achieved either through a honeycomb core or ribs (of which a nominal design might look like that illustrated here:
![Nominal ribs for a turntable](figures/nominal-turntable-ribs.png)

In [13]:
core_vf = 0.15 # honeycomb core volume fraction, typically about 8-32% depending on design
ribs_af = 0.20 # approx area fraction of some nominal ribs stiffening design for turntable plate

#### CFRP sandwich construction
CFRP would have a higher stiffness per unit mass than the metals, as well as some natural structural damping. We would likely use an intermediate modulus fiber in a quasi-isotropic, balanced, symmetric layup. Matrix would be 350$^o$F (450 K) cured resin such as Toray RS-3C, commonly used in spacecraft. The design would need to be sufficiently low-strain such that the total temperature change from processing (450 K) down to lunar night (100 K) does not exceed fiber strain limits. A honeycomb sandwich construction would be most mass-efficient, with titanium or aluminum inserts at bolted connections. Thermal expansion effects would be low, several parts per million (ppm) per degree K. Low out-of-plane thermal conductivity would be of some benefit in reducing the heat transfer rate from the turntable to the rest of the spacecraft below it. The main detractions of CFRP are the higher design and fabrication costs.

In [14]:
plate_sandwich_skin_t = 0.0016
plate_sandwich_core_t = plate_t - 2 * plate_sandwich_skin_t
plate_sandwich_skin_rho = cfrp_density
plate_sandwich_core_rho = rho_Al * core_vf
plate_sandwich_mass = (2*plate_sandwich_skin_rho*plate_sandwich_skin_t + plate_sandwich_core_rho*plate_sandwich_core_t) * plate_area
plate_masses = {'CFRP sandwich': plate_sandwich_mass}

#### Metal ribbed plate construction
Material would be either titanium or aluminum.

A machined aluminum plate would be the least expensive to fabricate. Detractions are the higher CTE ($\sim$12-25 ppm/K over the 100K-400K temperature range) and reduced stress allowables at sustained high temperature (7075-T651 aluminum, for example, may have yield strength reduction from $\sim$ 500 MPa at room temperature to $\sim$ 180 MPa when sustained at 150$^o$C (423 K). The high thermal conductivity of aluminum allows faster heat transfer toward the spacecraft. All these detractions can likely be mitigated with use of appropriate design allowables and thermal breaks.

Grade 5 titanium would be a robust solution. Material cost would be comparable to CFRP, though fabrication complexity would be simpler. CTE is relatively low and stable ($\sim$5-9 ppm/K over range 100K-400K), strength is high, and the very low thermal conductivity ($\sim$5-8 W/m*K) would provide some natural insulation of the rest of the spacecraft from the exposed turntable's temperature. For cost and ease of manufacturing we would probably not machine the plate out of thick titanium, but rather screw together a thinner face sheet with stiffening ribs.

In [15]:
plate_rhos = {'aluminum': rho_Al,
              'titanium': rho_Ti,
             }  # density
plate_face_t = 0.0016  # face plate thickness
plate_ribs_t = plate_t - plate_face_t
plate_volume = (plate_face_t + plate_ribs_t * ribs_af) * plate_area
for name, rho in plate_rhos.items():
    plate_masses[name] = rho * plate_volume

#### Plate construction options summary

In [16]:
plate_inertias = {}
for name, mass in plate_masses.items():
    plate_inertias[name] = mass * uniform_plate_inertia_over_mass  # note this uniform assumption is good for a typical evenly-spaced rib pattern
    print(f'Turntable mass = {plate_masses[name]:.3f} kg, inertia = {plate_inertias[name]:.4f} kg/m^2 ({name})')

Turntable mass = 1.566 kg, inertia = 0.0502 kg/m^2 (CFRP sandwich)
Turntable mass = 1.695 kg, inertia = 0.0543 kg/m^2 (aluminum)
Turntable mass = 2.782 kg, inertia = 0.0891 kg/m^2 (titanium)


In [17]:
# For purposes of motor sizing, select the heaviest option.
plate_mass = max(plate_masses.values())
plate_inertia = max(plate_inertias.values())

### Antennas
The four antennas will be stacer tubes, equally spaced around the turntable. Deployed length is 3 m for all antennas. Deployment angle is slightly above horizontal, counteracting gravity sag. Here this slight bend is neglected, and treated as purely horizontal.

In [18]:
boom_deployed_wall_in = 0.005  # inches, antenna approx wall thickness when deployed
boom_deployed_diam_in = 0.35  # inches, antenna approx diameter when deployed
boom_wall, boom_diam, boom_area, boom_length, boom_inertia = {}, {}, {}, {}, {}
boom_wall['deployed'] = boom_deployed_wall_in * m_per_in
boom_diam['deployed'] = boom_deployed_diam_in * m_per_in
boom_area['deployed'] = math.pi * boom_diam['deployed'] * boom_wall['deployed']
boom_length['deployed'] = 3.0
boom_volume = boom_length['deployed'] * boom_area['deployed']
boom_rho = rho_BeCu  # select material for antenna
boom_mass = boom_volume * boom_rho
print(f'Antenna boom mass = {boom_mass:.3f} kg each')

Antenna boom mass = 0.088 kg each


In [19]:
boom_length['stowed'] = 0.105  # assumption based on July 2022 CAD model
boom_area['stowed'] = boom_volume / boom_length['stowed']
boom_diam['stowed'] = boom_diam['deployed']  # approx guess
boom_wall['stowed'] = boom_area['stowed'] / (math.pi * boom_diam['stowed'])

Each antenna also has a tip piece. The mass and size are estimated here from the July 2022 CAD model.

In [20]:
ant_tip_mass = 0.029
ant_tip_length = 0.156
ant_tip_diam = boom_diam['deployed'] - boom_wall['deployed']
ant_tip_inertia_about_cg = ant_tip_mass/48 * (3*ant_tip_diam**2 + 4*ant_tip_length**2)
print(f'Antenna mass (boom + tip) = {boom_mass + ant_tip_mass:.3f} kg each')

Antenna mass (boom + tip) = 0.117 kg each


In [21]:
boom_root_radial_position = plate_od/2 - boom_length['stowed'] # where the antenna base end is, radially on the turntable
ant_inertia = {}
for k in ('deployed', 'stowed'):
    boom_OD = boom_diam[k] + boom_wall[k] / 2
    boom_ID = boom_diam[k] - boom_wall[k] / 2
    boom_inertia_about_cg = boom_mass/48 * (3*boom_OD**2 + 3*boom_ID**2 + 4*boom_length[k]**2)
    boom_offset = boom_root_radial_position + boom_length[k] / 2
    ant_tip_offset = boom_root_radial_position + boom_length[k] - ant_tip_length/2
    boom_inertia = boom_inertia_about_cg + boom_mass * boom_offset**2
    ant_tip_inertia = ant_tip_inertia_about_cg + ant_tip_mass * ant_tip_offset**2
    ant_inertia[k] = boom_inertia + ant_tip_inertia
    print(f'Antenna inertia (boom + tip) = {ant_inertia[k]:.4f} kg/m^2 each ({k})')

Antenna inertia (boom + tip) = 0.5781 kg/m^2 each (deployed)
Antenna inertia (boom + tip) = 0.0044 kg/m^2 each (stowed)


### Antenna deployers
As of July 2022, the deployers and attached hardware (including pre-amp?) are assumed to be $\sim$ 0.75 kg per antenna. This value will change somewhat as the design is refined.

For the purposes of this calculation, the deployer geometry is treated as a cuboid with side lengths estimated based on the stowed antenna length.

In [22]:
deployer_mass = 0.75
deployer_length = boom_length['stowed'] * 3
deployer_width = boom_length['stowed']
deployer_radial_position = plate_od / 2
deployer_inertia_about_cg = deployer_mass / 12 * (deployer_width**2 + deployer_length**2)
deployer_inertia = deployer_inertia_about_cg + deployer_mass * deployer_radial_position**2
print(f'Deployer mass = {deployer_mass:.3f} kg (each)')
print(f'Deployer inertia = {deployer_inertia:.4} kg/m^2 (each)')

Deployer mass = 0.750 kg (each)
Deployer inertia = 0.05377 kg/m^2 (each)


### Central shaft and bearings
These rotating components are lumped together and approximated as a single tube of metal.

In [23]:
central_od = 0.080  # outer diameter
central_id = 0.070  # inner diameter
central_length = 0.020  # length
central_r1, central_r2 = central_id/2, central_od/2  # radii
central_rho = rho_SS
central_mass = central_rho * central_length * math.pi * (central_r2**2 - central_r1**2)
central_inertia = 0.5 * central_mass * (central_r1**2 + central_r2**2)
print(f'Central shaft and bearing mass = {central_mass:.3f} kg')
print(f'Central shaft and bearing inertia = {central_inertia:.4} kg/m^2')

Central shaft and bearing mass = 0.188 kg
Central shaft and bearing inertia = 0.0002662 kg/m^2


### Parallel-axis transmission
The motor will be mounted off-axis and parallel to the central bearing.

In the present study (July 2022), details of this transmission are not considered. For code completeness, a gear ratio is included, but here set to 1:1. A simple torque reduction factor is assumed, accounting for frictional loss in this transmission.

In [24]:
# Note: The transmission ratio here does *not* include the internal gearing of the motor.
# This value is only about the transmission between it and the turntable.
transmission_ratio = 1.0 / 1.0  # gearmotor output revolutions per turntable revolution
transmission_efficiency = 0.90  # assumption based on typical roller chain drives

### Motor shaft, gearhead, bearing
In the present study (July 2022), whose initial purpose is to define specifications for the motor, the internal details of the motor are not considered. They could be added to this study at a later date, after we have selected a specific motor model.

### External dust seal

Some dust seal(s) will be required to prevent lunar dust from accessing the bearing and transmission. These impose some friction load on the assembly.

Here I assume the seal contact is made against a flat face by annular, thin-walled polyimide. In this approach, there would be one seal at the turntable rotation axis, and another at the motor output, covering the off-axis transmission components. (The motor may additionally have internal seals in its bearing at the output shaft, TBD.)

The seal at the central axis will bear either on the rotating turntable, or else be mounted to the turntable and bear on its opposing counterpart (i.e. the top plate or the central shaft support). Preload force will be relatively low, since only dust needs to be excluded.

Some comments are offered in another section about the option of alternatively, or additionally, having internal seals integral to the bearing. There are both advantages and disadvantages. It would be necessary for the bearing vendor to fabricate using a seal material with which we are comfortable at the lunar temperature extremes and in vacuum.

In [25]:
seal_contact_radius = 0.080  # annulus of contact for the seals
seal_contact_width = 0.003  # rough guess as to desirable contact width, will depend on seal design details
seal_contact_pressure_millibar = 10  # rough guess as to desirable clamping pressure (per seal)
seal_contact_pressure = seal_contact_pressure_millibar * 100
seal_contact_area = 2 * math.pi * seal_contact_radius * seal_contact_width
seal_clamping_force = seal_contact_pressure * seal_contact_area
print(f'External dust seal at radius {seal_contact_radius:.3f} m: clamping force = {seal_clamping_force:.1f} N')

External dust seal at radius 0.080 m: clamping force = 1.5 N


Datasheet values for the static and dynamic friction coefficient of [Vespel SP-1](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-1) are listed at 0.29 and 0.35, respectively. I presume a fixed, static value of 0.35 for the purposes of this calculation, and estimate a fixed torque based on the assumed contact annulus.

In [26]:
seal_mu = 0.35
seal_torque = seal_mu * seal_contact_radius * seal_clamping_force
print(f'External dust seal static torque = {seal_torque:.3f} N*m = {seal_torque / Nm_per_ozin:.1f} oz*in')

External dust seal static torque = 0.042 N*m = 6.0 oz*in


### Bearing friction
For the purposes of preliminary overall system torque calculations, I presume a bearing with its own integrated dust seal. This may not be the case, as discussed in more detail in the bearing selection section below. But presently, in sizing the motor and transmission, I assume selection of a single Type-X, nitrile double-sealed, stainless steel bearing, e.g. Kaydon WA025XP0. The vendor provides a static torque value for this bearing, given default lubrication and temperature conditions:

In [27]:
bearing_torque_ozin = 8.0  # oz*in
bearing_torque = bearing_torque_ozin * Nm_per_ozin
print(f'Central bearing static torque = {bearing_torque:.3f} N*m')

Central bearing static torque = 0.056 N*m


### Harness
The primary elastic contributor to motor driving torque is the cable harness. Its torsional stiffness is estimated here by counting the number of conductors, assuming a diameter for each conductor, and then adding up their torsional stiffness plus their insulating coatings in parallel. The individual conductors are here treated as solid core, though in reality they would be stranded.

In [28]:
wires_per_antenna = 20
n_wires = n_ant * wires_per_antenna
wire_diam_mm = 0.35
wire_insul_wall_mm = 0.15  # wall thickness of insulator on wire
wire_diam = wire_diam_mm / 1000
wire_insul_wall = wire_insul_wall_mm / 1000
J_wire = math.pi/32 * wire_diam**4  # wire polar moment of inertia
J_insul = math.pi/32 * ((wire_diam + 2*wire_insul_wall)**4 - wire_diam**4)  # insulation polar moment of inertia
harness_length = 0.3  # estimated from July 2022 CAD model
shear_modulus = lambda E: E / (2*(1 + 0.3))  # approx shear modulus for typical poisson ratio
insulated_wire_stiffness = (shear_modulus(E_Cu) * J_wire + shear_modulus(E_plastic) * J_insul) / harness_length  # N*m / rad
harness_stiffness = n_wires * insulated_wire_stiffness
harness_max_torque = harness_stiffness * max(rotation_range)
for unit, scale in {'N*m/rad': 1.0, 'N*mm/deg': 180/math.pi*1000, 'oz*in/deg': 180/math.pi/Nm_per_ozin}.items():
    print(f'Harness approx torsional stiffness = {harness_stiffness*scale:.4g} {unit}')
for unit, scale in {'N*m': 1.0, 'N*mm': 1000., 'oz*in': 1/Nm_per_ozin}.items():
    print(f'Harness max torque resistance = {harness_max_torque*scale:.4g} {unit}')


Harness approx torsional stiffness = 0.02097 N*m/rad
Harness approx torsional stiffness = 1202 N*mm/deg
Harness approx torsional stiffness = 170.2 oz*in/deg
Harness max torque resistance = 0.03477 N*m
Harness max torque resistance = 34.77 N*mm
Harness max torque resistance = 4.924 oz*in


## Performance
Our needs for rotational speed and acceleration are relatively low. 

### Speed
Repositioning will take on the order of minutes. We base the speed on the anticipated control feedback loop speed provided by the CD&H computer, and on our desired positioning precision.

In [29]:
feedback_rate = 1.0  # Hz
precision_deg = 1.0  # deg
precision = math.radians(precision_deg)
max_allowable_speed = precision * feedback_rate
nom_operating_speed_rpm = 0.1
nom_operating_speed = nom_operating_speed_rpm * radsec_per_rpm
antibacklash_move_deg = 3.0
antibacklash_move = math.radians(antibacklash_move_deg)
max_slew_distance = total_rotation_range + antibacklash_move
max_slew_time = max_slew_distance / nom_operating_speed
print(f'Max slew time = {max_slew_time:5.1f} sec = {max_slew_time/60:4.1f} min at nominal operating speed '
      f'({nom_operating_speed_rpm:.3g} rpm = {math.degrees(nom_operating_speed):.2f} deg/s).')
print(f'Maximum allowable speed = {max_allowable_speed/radsec_per_rpm:.3g} rpm = {math.degrees(max_allowable_speed):.1f} deg/s.')
assert max_allowable_speed >= nom_operating_speed, f'Must define a nominal operating speed lower than max_allowable_speed'

Max slew time = 321.7 sec =  5.4 min at nominal operating speed (0.1 rpm = 0.60 deg/s).
Maximum allowable speed = 0.167 rpm = 1.0 deg/s.


### Acceleration
Acceleration must not exceed the buckling strength of the extended antenna. Here we estimate the buckling critical load per Roark's 7th Edition:

![Transverse buckling of thin-walled tube](figures/roarks_7th_tube_transverse_buckling.png)

In [30]:
buckling_design_safety_factor = 100  # stay conservative, since this is both Euler instability and since inertias / motor details not yet fully defined
tube_transverse_buckling_K = 0.72  # using minimum value here, to stay on conservative side
boom_root_radial_position = plate_od
boom_critical_moment = tube_transverse_buckling_K * E_BeCu / (1 - nu_BeCu**2) * boom_length['deployed'] * boom_wall['deployed']**2
boom_allowable_moment = boom_critical_moment / buckling_design_safety_factor
boom_allowable_accel = boom_allowable_moment / ant_inertia['deployed']
boom_allowable_accel_period = nom_operating_speed / boom_allowable_accel
print(f'Antenna critical buckling moment = {boom_critical_moment:.1f} N*m')
print(f'Applying safety factor of {buckling_design_safety_factor} --> allowable moment = {boom_allowable_moment:.1f} N*m')
print(f'--> max allowable acceleration = {boom_allowable_accel:.1f} rad/s^2 = {math.degrees(boom_allowable_accel):.1f} deg/s^2 = {boom_allowable_accel/(2*math.pi):.1f} rev/s^2')
print(f'--> min allowable period of acceleration = {boom_allowable_accel_period:.3g} sec')

Antenna critical buckling moment = 4976.9 N*m
Applying safety factor of 100 --> allowable moment = 49.8 N*m
--> max allowable acceleration = 86.1 rad/s^2 = 4932.8 deg/s^2 = 13.7 rev/s^2
--> min allowable period of acceleration = 0.000122 sec


We assert a range for the nominal acceleration period based on assuring a practical spin-up period. For completeness, the buckling criterion acceleration is checked as well, though it is generally much faster than we expect any practical real-world system to perform. A wide range of allowable acceleration performance is thus defined.

In [31]:
nom_accel_period_max = 5.0  # sec, slow end of practical performance
practical_accel_period_min = 0.01  # sec, fast end of practical performance
nom_accel_period_min = max(boom_allowable_accel_period, practical_accel_period_min)
nom_accel = [nom_operating_speed / t for t in [nom_accel_period_max, nom_accel_period_min]]
print(f'Nominal bounds on system acceleration rate = ({min(nom_accel):.3g}, {max(nom_accel):.3g}) rad/s^2')
assert max(nom_accel) < boom_allowable_accel, f'nominal acceleration is higher than boom can tolerate'

Nominal bounds on system acceleration rate = (0.00209, 1.05) rad/s^2


## Torque margin calculation

### Resistive known torques

In [32]:
total_inertia = plate_inertia + n_ant * (ant_inertia['deployed'] + deployer_inertia) + central_inertia
T_known = [total_inertia * a for a in nom_accel]
print(f'Total payload inertia (i.e. not including motor) = {total_inertia:.4g} kg*m^2')
print(f'Range for total "known" resistive torque = ({min(T_known):.3g}, {max(T_known):.3g}) N*m')

Total payload inertia (i.e. not including motor) = 2.617 kg*m^2
Range for total "known" resistive torque = (0.00548, 2.74) N*m


### Resistive variable torques

In [33]:
T_static_friction = [bearing_torque, seal_torque + bearing_torque]  # range of options, since final sealing config not yet determined
T_transmission_loss = [(1 - transmission_efficiency) * (min(T_static_friction) + min(T_known)),
                       (1 - transmission_efficiency) * (max(T_static_friction) + max(T_known))]
T_variable = [min(T_static_friction) + min(T_transmission_loss) + harness_max_torque,
              max(T_static_friction) + max(T_transmission_loss) + harness_max_torque]
print(f'Range for static friction torque = ({min(T_static_friction):.3g}, {max(T_static_friction):.3g}) N*m')
print(f'Range for transmission loss = ({min(T_transmission_loss):.3g}, {max(T_transmission_loss):.3g}) N*m')
print(f'Range for "variable" resistive torque = ({min(T_variable):.3g}, {max(T_variable):.3g}) N*m')

Range for static friction torque = (0.0565, 0.0987) N*m
Range for transmission loss = (0.0062, 0.284) N*m
Range for "variable" resistive torque = (0.0975, 0.417) N*m


### Minimum driving torque
We use the resistive known and variable torques, as well as the GEVS safety factors, to determine the minimum allowable motor torque. We apply the safety factors to the minimum allowable acceleration combined with the maximum friction.

In [34]:
TM = 0.0  # set to 0 for minimum allowable, i.e. positive torque margin
T_avail_min = (TM + 1) * (FSk * min(T_known) + FSv * max(T_variable))
print(f'For positive torque margin, with safety factors FSk = {FSk} and FSv = {FSv},\n'
      f'minimum gearmotor torque shall be T_avail = {T_avail_min:.3g} N*m = {T_avail_min/Nm_per_ozin:.3g} oz*in')

For positive torque margin, with safety factors FSk = 2.0 and FSv = 4.0,
minimum gearmotor torque shall be T_avail = 1.68 N*m = 238 oz*in


### Maximum driving torque
Because we may not be allowed speed control by the provided electronics in the LuSEE-Night system, we wish to target a torque which is not only too small, but also not too large. Thus to set a maximum allowable torque, we sum the maximum allowable acceleration and minimum friction case, applying no safety factors.

In [35]:
T_avail_max = max(T_known) + min(T_variable)
print(f'To keep acceleration within required limits,\n'
      f'maximum gearmotor torque shall be T_avail = {T_avail_max:.3g} N*m = {T_avail_max/Nm_per_ozin:.3g} oz*in')

To keep acceleration within required limits,
maximum gearmotor torque shall be T_avail = 2.84 N*m = 402 oz*in


## Appendix: Motor selection
As of July 2022, we have been presuming usage of Globe Motor A-1430. This model indeed is offered with reduction ratios that deliver 300 oz-in of torque. Therefore we can proceed further with this motor family.

At 12 VDC, the Globe Motors have two armature winding options:

In [36]:
globe_armature = {'-15': {'no load speed (rpm)': [13500, 17000],
                          'max rated torque (oz*in)': 0.22,
                          'Kt (oz*in/A)': 0.95,
                          'R': 3.70,  # ohms
                         },
                  '-14': {'no load speed (rpm)': [10000, 13000],
                          'max rated torque (oz*in)': 0.33,
                          'Kt (oz*in/A)': 1.32,
                          'R': 6.46,  # ohms
                         },
                 }

Among enclosed type motors, the family includes 7 options with maximum continous torque of 300 oz\*in:

In [50]:
globe_gear_max_continuous_torque_ozin = 300.
globe_gear = {'43A152': {'speed reduction ratio': 2799,
                         'torque multiplier ratio': 900,
                         'length (in)': 3.28,
                        },
              '43A153': {'speed reduction ratio': 4230,
                         'torque multiplier ratio': 1400,
                         'length (in)': 3.28,
                        },
              '43A154': {'speed reduction ratio': 6391,
                         'torque multiplier ratio': 2100,
                         'length (in)': 3.28,
                        },
              '43A155': {'speed reduction ratio': 10689,
                         'torque multiplier ratio': 2800,
                         'length (in)': 3.45,
                        },
              '43A156': {'speed reduction ratio': 16150,
                         'torque multiplier ratio': 4200,
                         'length (in)': 3.45,
                        },
              '43A157': {'speed reduction ratio': 24403,
                         'torque multiplier ratio': 6400,
                         'length (in)': 3.45,
                        },
              '43A158': {'speed reduction ratio': 36873,
                         'torque multiplier ratio': 9700,
                         'length (in)': 3.45,
                        },
             }

{'43A152': {'speed reduction ratio': 2799,
  'torque multiplier ratio': 900,
  'length (in)': 3.28},
 '43A153': {'speed reduction ratio': 4230,
  'torque multiplier ratio': 1400,
  'length (in)': 3.28},
 '43A154': {'speed reduction ratio': 6391,
  'torque multiplier ratio': 2100,
  'length (in)': 3.28},
 '43A155': {'speed reduction ratio': 10689,
  'torque multiplier ratio': 2800,
  'length (in)': 3.45},
 '43A156': {'speed reduction ratio': 16150,
  'torque multiplier ratio': 4200,
  'length (in)': 3.45},
 '43A157': {'speed reduction ratio': 24403,
  'torque multiplier ratio': 6400,
  'length (in)': 3.45},
 '43A158': {'speed reduction ratio': 36873,
  'torque multiplier ratio': 9700,
  'length (in)': 3.45}}

The combinations are evaluated, filtered, and sorted by power consumption.

In [59]:
gearmotor = {}
for a, arm in globe_armature.items():
    for g, gear in globe_gear.items():
        part = f'{g}{a}'
        gearmotor[part] = {}
        gear['min gear efficiency'] = gear['torque multiplier ratio'] / gear['speed reduction ratio']
        gearmotor[part]['rated output torque (oz*in)'] = arm['max rated torque (oz*in)'] * gear['torque multiplier ratio']
        gearmotor[part]['gearhead torque limit safety factor'] = globe_gear_max_continuous_torque_ozin / gearmotor[part]['rated output torque (oz*in)']
        gearmotor[part]['current'] = arm['max rated torque (oz*in)'] / arm['Kt (oz*in/A)']
        gearmotor[part]['power'] = gearmotor[part]['current']**2 * arm['R']
# calcs above need a double-check --- I typed them quickly
globe_gear

{'43A152': {'speed reduction ratio': 2799,
  'torque multiplier ratio': 900,
  'length (in)': 3.28,
  'min gear efficiency': 0.3215434083601286},
 '43A153': {'speed reduction ratio': 4230,
  'torque multiplier ratio': 1400,
  'length (in)': 3.28,
  'min gear efficiency': 0.3309692671394799},
 '43A154': {'speed reduction ratio': 6391,
  'torque multiplier ratio': 2100,
  'length (in)': 3.28,
  'min gear efficiency': 0.32858707557502737},
 '43A155': {'speed reduction ratio': 10689,
  'torque multiplier ratio': 2800,
  'length (in)': 3.45,
  'min gear efficiency': 0.26195153896529144},
 '43A156': {'speed reduction ratio': 16150,
  'torque multiplier ratio': 4200,
  'length (in)': 3.45,
  'min gear efficiency': 0.26006191950464397},
 '43A157': {'speed reduction ratio': 24403,
  'torque multiplier ratio': 6400,
  'length (in)': 3.45,
  'min gear efficiency': 0.2622628365364914},
 '43A158': {'speed reduction ratio': 36873,
  'torque multiplier ratio': 9700,
  'length (in)': 3.45,
  'min gear

In [54]:
gearmotor

{'43A152-15': {'rated output torque (oz*in)': 198.0,
  'gearhead torque limit safety factor': 1.5151515151515151,
  'current': 0.23157894736842105,
  'power': 0.19842659279778396},
 '43A153-15': {'rated output torque (oz*in)': 308.0,
  'gearhead torque limit safety factor': 0.974025974025974,
  'current': 0.23157894736842105,
  'power': 0.19842659279778396},
 '43A154-15': {'rated output torque (oz*in)': 462.0,
  'gearhead torque limit safety factor': 0.6493506493506493,
  'current': 0.23157894736842105,
  'power': 0.19842659279778396},
 '43A155-15': {'rated output torque (oz*in)': 616.0,
  'gearhead torque limit safety factor': 0.487012987012987,
  'current': 0.23157894736842105,
  'power': 0.19842659279778396},
 '43A156-15': {'rated output torque (oz*in)': 924.0,
  'gearhead torque limit safety factor': 0.3246753246753247,
  'current': 0.23157894736842105,
  'power': 0.19842659279778396},
 '43A157-15': {'rated output torque (oz*in)': 1408.0,
  'gearhead torque limit safety factor': 0.

## Appendix: Bearings selection

### Bearing type and size
The preliminary design (July 2022) packaging acommodates a pair of Kaydon KA025AR0 bearings:

![Kaydon KA0xxAR0](figures/kaydon-KA0xxAR0-open-A-bearings.png)

This is a thin section, angular contact, open bearing, with bore 2.5\" (63.5 mm), O.D. 3.0\" (76.2 mm), and thickness 0.25\" (6.35 mm). Mass is low, ~0.12 lbs = 55 g. We utilize the common design pattern of mounting in pairs, to provide tilt resistance.

Stainless steel options (may have custom lead-times and will be more expensive) are available. Replace "K" with "S" in the part number.

Kaydon also offers an attractive "Type X" four-point contact bearing. We have previously used such a bearing (at a much larger scale) in the Dark Energy Spectroscopic Instrument (DESI) ADC rotator mechanism. The type X controls 5 degrees of freedom in a single unit, rather than the typical angular contact duplex. This has the advantage of fewer parts and less weight, as well as removing the need to control thrust preload in the mounting between the two separate bearing units. A type X unit of similar package size as KA025AR0 would be KA025XP0:

![Kaydon KA0xxXP0](figures/kaydon-KA0xxXP0-open-X-bearings.png)

Again, replace "K" with "S" for equivalent stainless option.

### External vs internal bearing seals

For these open bearings, we would make a dust seal that rides on the shaft and/or flat face. Call this an "external" seal. It has the advantages of being:

- independent from the bearing selection and procurement
- easily replaceable
- visible and inspectable
- material of our choice (i.e. robust, low cte, temperature tolerant material like polyimide)

We can alternatively purchase a bearing with integrated seals. Call this an "internal" seal. It has the advantages of:

- lower part count
- lower mass
- more compact packaging

Again taking Kaydon as an example bearing manufacturer, their standard seal material is nitrile rubber. This would need to be qualified for our temperature range (100 K - 400 K). Kaydon does offer customization options in alternate seal materials, but we have not yet checked whether polyimide in particular would work for them. Several models Kaydon offers with an integrated double-seal are given in this table:

![Kaydon JA0xxXP0](figures/kaydon-JA0xxXP0-sealed-X-bearings.png)

The Kaydon line-up does not have sealed bearings in angular contact configurations. 

For sealed bearings, replace "J" with "W" in the part number for the stainless option.

### Bearing seal materials

If we fabricate our own seals, one might consider using [Vespel SP-3](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-3). This is a 15% MoS<sub>2</sub>-filled polyimide with a long history of use in space applications, with low creep, high strength, and low coefficient of thermal expansion (for a polymer), and which should survive the temperature extremes.

A concern I have is that the friction coefficient of SP-3 in vacuum is reported to be very low (0.03) in the Dupont datasheet. Normally lower friction is "good" in a bearing, however in our design, having some consistent friction is helpful, to guarantee stability of the position of the turntable stable when powered off. So we probably rather prefer having a consistent friction whether testing in lab air or operating in vacuum. In this respect, unfilled [Vespel SP-1](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-1) may be a simpler and better choice. However, the DuPont datasheet does not specifically list a vacuum friction coefficient, so this bears some further investigation.

Graphite-filled polyimide grades would not be chosen, since the graphite lubricant can in fact become abrasive in a moisture-free environment.

### Turntable bearings

There are some bearings available with more integrated turntable functionality, for example including an external gear profile. An attractive option may be Kaydon T01-00325EAA:

![Kaydon turntables - basic](figures/kaydon-turntable-bearings-basic.png)

![Kaydon turntables - external gears](figures/kaydon-turntable-bearings-extgear.png)

![Kaydon turntables - illustrations](figures/kaydon-turntable-illustrations.png)

These turntables are not off-the-shelf parts. Cost and lead-time would be TBD. Performance is not particularly any better than the other options; the attraction here would be the compact packaging. The gearing would need some exterior protection from lunar dust.

## References
- [NASA General Environmental Verification Standards (GEVS)](https://standards.nasa.gov/sites/default/files/standards/GSFC/B/0/gsfc-std-7000b_signature_cycle_04_28_2021_fixed_links.pdf)
- [Kaydon turntable bearings](https://www.kaydonbearings.com/RealiSlim_TT_bearings.htm)
- [Kaydon sealed, slim bearings](https://www.kaydonbearings.com/RealiSlim_sealed_bearings.htm)